In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.models import load_model

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
# Split the training set into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [4]:
print('Train Images Shape:      ', X_train.shape)
print('Train Labels Shape:      ', y_train.shape)

print('\nValidation Images Shape: ', X_val.shape)
print('Validation Labels Shape: ', y_val.shape)

print('\nTest Images Shape:       ', X_test.shape)
print('Test Labels Shape:       ', y_test.shape)

Train Images Shape:       (40000, 32, 32, 3)
Train Labels Shape:       (40000, 1)

Validation Images Shape:  (10000, 32, 32, 3)
Validation Labels Shape:  (10000, 1)

Test Images Shape:        (10000, 32, 32, 3)
Test Labels Shape:        (10000, 1)


In [5]:
# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Print shapes to verify
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (40000, 32, 32, 3)
X_val shape: (10000, 32, 32, 3)
X_test shape: (10000, 32, 32, 3)


In [6]:
# Transform target variable into one-hotencoding
y_category_train = to_categorical(y_train, 10)
y_category_val = to_categorical(y_val, 10)
y_category_test = to_categorical(y_test, 10)

In [ ]:
data_generator = ImageDataGenerator(
    width_shift_range=0.1,

    rotation_range=10,

    # zoom_range=0.1,

    # brightness_range=[0.9,1.1],

    # shear_range=10,

    # channel_shift_range=0.1,

    height_shift_range=0.1, 

    horizontal_flip=True    
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

In [ ]:
from keras.applications import ResNet50
from keras.layers import Dense
from keras.models import Sequential

batch_size = 32
train_generator = data_generator.flow(X_train, y_category_train, batch_size)
steps_per_epoch = X_train.shape[0] // batch_size

model = Sequential()
base_model = ResNet50(input_shape=(32, 32, 3), include_top=False, weights='imagenet', pooling='avg')
model.add(base_model)
model.add(Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

r = model.fit(train_generator, 
              epochs=20,
              steps_per_epoch=steps_per_epoch,
              validation_data=(X_val, y_category_val), 
              callbacks=[early_stop],
             )

94765736/94765736 [==============================] - 7s 0us/step
Epoch 1/50
  55/1250 [>.............................] - ETA: 17:53 - loss: 4.4611 - accuracy: 0.1148

KeyboardInterrupt: 